In [ ]:
import sys
import os 
import open3d as o3d
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0
sys.path.append(os.path.join(os.getcwd(), ".."))
%cd ..
from generate import *
from utils.config import cfg_from_yaml_file
from joblib import Parallel, delayed
    

def remove_preexisting(sample_path):
    if os.path.exists(sample_path):
        [os.remove(os.path.join(sample_path, path)) for path in os.listdir(sample_path)]
    if not os.path.exists(sample_path):
        os.mkdir(sample_path)
    plt.show()

In [ ]:
config = cfg_from_yaml_file("custom/generator_config.yaml")
print(config)
forests = os.listdir("data/forest/original")
print(forests)

In [ ]:
N_SAMPLES = 100
VERBOSE = True
def process(forest, config):
    config.plot_path = forest
    config.sample_path = os.path.join("data/chunks/",forest.split("/")[-1])
    #remove_preexisting(config.sample_path)
    config.occupancy_path = os.path.join("data/forest/occupancy", forest.split("/")[-1])
    generator = SampleGenerator(config)
    generator.occupancy = generator.occupancy_grid()
    vertices, boolean, rotations, centers, vertices_unrotated = generator.random_generate(N_SAMPLES)
    if VERBOSE:
        plotfn(vertices, boolean, generator)
    samples, centers, rotations = generator.produce_samples_cuda(vertices[boolean], rotations[boolean], centers[boolean], vertices_unrotated[boolean])
    paths = generator.save_samples(samples, centers, rotations)
    print(len(paths), "saved for forest", forest)
    #return samples, centers, rotations
    return None

In [ ]:
# for forest in forests:
#     sample_path = os.path.join("data/chunks", forest)
#     if len(os.listdir(sample_path)) < 500:
#         print("process ", forest)
#         process(forest,config)

In [ ]:
def filter_cloud(source, target):
    points = np.load(source)
    #print(len(points))
    points, label = points[:, :3], points[:,3]
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    
    # filter out point
    cl, ind = pcd.remove_statistical_outlier(nb_neighbors=2,  std_ratio=1)
    pcd = pcd.select_by_index(ind)
    mask = np.zeros(len(points),dtype=bool)
    mask[ind] = True
    points = points[mask]
    label = label[mask]
    #print(len(points))
    cl, ind = pcd.remove_radius_outlier(nb_points=6, radius=0.2068)
    pcd = pcd.select_by_index(ind)
    mask = np.zeros(len(points),dtype=bool)
    mask[ind] = True
    points = points[mask]
    #print(len(points))
    points = np.hstack((points, label[mask][:, np.newaxis]))
    np.save(target, points)
    #print(target)
      

In [ ]:
for forest in forests:
    save_path = os.path.join("data/chunks", forest)
    source_path = os.path.join("data/chunks_notfiltered", forest)
    existing_trees = os.listdir(source_path)
    source_paths = [os.path.join(source_path, tree) for tree in existing_trees]
    save_paths = [os.path.join(save_path, tree) for tree in existing_trees]
    print(forest, len(save_paths))
    Parallel(n_jobs=16, verbose=1)(delayed(filter_cloud)(source, target) for source, target in zip(source_paths, save_paths))

In [ ]:
for forest in forests:
    sample_path = os.path.join("data/chunks", forest)
    print(len(os.listdir(sample_path)))

In [ ]:
for forest in forests:
    sample_path = os.path.join("data/chunks", forest)
    existing_trees = os.listdir(sample_path)
    existing_trees = [tree if tree.split(".")[-1] == "txt" else None for tree in existing_trees]
    existing_trees = np.array(existing_trees)
    filtered = existing_trees[existing_trees != None]
    target_paths = [os.path.join(sample_path, tree) for tree in filtered]
    [os.remove(path) for path in target_paths]